In [1]:
import sys, os
sys.path.append(os.pardir)
from common.tools import *
from bs4 import BeautifulSoup
import urllib.request as req
import numpy as np
from time import sleep
import math
import csv
import requests
import datetime

date = datetime.datetime.now()
tstamp = '%04d%02d%02d%02d%02d%02d' % (date.year, date.month ,date.day,date.hour,date.minute,date.second)

ROOT_URL = "https://www.1999.co.jp"
URL = "https://www.1999.co.jp/search?typ1_c=101&cat=&state=&sold=0&sortid=0&searchkey=%E8%81%96%E9%97%98%E5%A3%AB%E6%98%9F%E7%9F%A2&spage="
#URL= "https://www.1999.co.jp/search?typ1_c=101&cat=figure&state=&sold=0&sortid=0&searchkey=%e8%81%96%e9%97%98%e5%a3%ab%e6%98%9f%e7%9f%a2"
OUT_F = 'data/figure_items_%s.csv' % tstamp
LOG_F  = r".\log\log%s.log" % tstamp
res = req.urlopen(URL)
soup = BeautifulSoup(res,'html.parser')
NUM_PER_PAGE = 40
SLEEP_SEC =1

# Request URL & get html
def get_soup(url, sleep_sec):

    try:
        # Not Local 
        res = req.urlopen(url)
        soup = BeautifulSoup(res, 'html.parser')
        return soup

    except urllib.error.HTTPError as http_e:
        print( http_e.code , http_e.reason, url)
        log_str = str( http_e.code) + http_e.reason +  url
        write_log(LOG_F, str(http_e.code)  )
    except Exception as e:
        print(e)
    finally:
        sleep(sleep_sec)
        

In [16]:
def get_name(soup):
    try:
        name = soup.find('h2',class_='h2_itemDetail').text
    except:
        name =''
    finally:
        return name
    
def get_price(soup):
    try:
        soup = soup.find('tr',id="masterBody_trPrice")
        #print(soup)
        price = soup.find('span',class_="Price_Dai").text
        if '¥' in price:
            price = price.replace("\n","")
            price = price.replace("\r","")
            price = price.replace(" ","")
            price = price.split('(税')[0]
            price = price.replace("¥","").replace(",","")                    
    except:
        price = ''
    finally:
        return price

def get_price_normal(soup):
    try:
        soup = soup.find('tr',id="masterBody_trPriceNormal")
        for td_soup in soup.findAll('td'):
                price_normal = td_soup.text
                if '¥' in price_normal:
                    price_normal = price_normal.split('(税')[0]
                    price_normal = price_normal.replace("¥","").replace(",","")                    
    except:
        price_normal = ''
    finally:
        return price_normal


def get_jan_code(soup):
    try:
        soup = soup.find('tr',id="masterBody_trJanCode")
        for jtest2 in soup.findAll('td'):
            jan_code = jtest2.text
            if jan_code.isdecimal():
                return jan_code
    except:
        jan_code = ""
    finally:
        return jan_code

def get_hight(soup):
    try:
        for div in item_soup.find("div", id="masterBody_pnlItemExp").findAll("div"):
            if  "●全高：" in div.text:
                hight = div.text.split("●全高：")[1].split("\n")[0].replace("\n","").replace("\r","")[:20]
    except:
        print(e)
        hight = ""
    finally:
        return hight

def get_size(soup):
    size =""
    try:
        for div  in soup.find("div", class_="txt14px").findAll("div",class_=""):
            size_str = div.text.replace("\r\n","")
            if "●サイズ：" in size_str:
                size = size_str.split("●サイズ：")[1]
    except:
        pass
    finally:
            return size

        
def get_img(img_show_url, out_dir):
    img_url = get_soup(img_show_url,SLEEP_SEC).find("table", id="imgTarget").find("img")["src"]
    
    print(img_url)
    output_img_path = img_url.replace("/","_")
    img_url = ROOT_URL + img_url
    r = requests.get(img_url)
    if os.path.exists(out_dir) == False:
            os.mkdir(out_dir)
    with open( out_dir + output_img_path,'wb') as file: 
        file.write(r.content)
  

In [11]:
def get_maker(soup):
    maker = soup.find("a").text
    return maker

def get_details(soup):
    item_details_soup = soup.find("table", id="tblItemInfo")
    maker = scale = material = prototype = series = original = rellease_date = ""
    for td in item_details_soup.findAll("tr" ): 
        attri = td.text.replace("\n","").replace(" ","")
        try:
            if "メーカー：" in attri:
                maker = attri.replace("メーカー：","")
            elif "スケール：" in attri:
                scale = attri.replace("スケール：","")
            elif "素材：" in attri:
                material = attri.replace("素材：","")
            elif "原型制作：" in attri:
                prototype = attri.replace("原型制作：","")
                print("test")
            elif "シリーズ：" in attri:
                 series = attri.replace("シリーズ：","")
            elif "原作：" in attri:
                original  = attri.replace("原作：","")
            elif "発売" in attri and "日" in attri:
                 release_date = attri.split("：")[1]
        except Exception as e:
            print(e)
    
    details  = [maker, scale, material, prototype, series, original, release_date]
    return details


In [12]:
soup = get_soup(URL, SLEEP_SEC)

searched_num = soup.find('div',class_='list_kensu02').text.split(" ")[0]
searched_product_num = int(searched_num)
searched_page_num = math.ceil(searched_product_num / NUM_PER_PAGE)

In [17]:
for i in range(1,searched_page_num+1):
    print("page No = %s" % i)
    page_url = URL   + str(i)
    page_soup = get_soup(page_url, SLEEP_SEC)
    
    for produ_nm in page_soup.findAll('div','a',class_='ListItemName'):
        produ_id = produ_nm.find('a').get("href")
        item_url = ROOT_URL + produ_id
        item_soup = get_soup(ROOT_URL + produ_id,SLEEP_SEC)
        name  = get_name(item_soup)
        price = get_price(item_soup) 
        price_normal = get_price_normal(item_soup) 
        jan_code = get_jan_code(item_soup)
        maker, scale, material, prototype, series, original, release_date = get_details(item_soup)
        hight = get_hight(item_soup)
        size = get_size(item_soup)
        item_info = [produ_id,name, price,price_normal, jan_code, item_url, maker, scale, material, prototype, series, original, release_date,hight,size]
        write_info(OUT_F,item_info)
        img_list_soup = item_soup.find("div", id="ImageDetail")
        for url in img_list_soup.findAll('a'):    
            img_show_url = url.get("href" )
            get_img(ROOT_URL + img_show_url,"./data/images/%s/" % produ_id)
 

## Dev

In [ ]:
ITEM_URL = "https://www.1999.co.jp/10679272" 
item_soup = get_soup(ITEM_URL, 1)
img_list_soup = item_soup.find("div", id="ImageDetail")
for url in img_list_soup.findAll('a'):    
    img_show_url = url.get("href" )
    get_img(ROOT_URL + img_show_url,"./data/")

In [60]:
#soup = get_soup(URL,SLEEP_SEC)
r = requests.get(URL)
with open('./data/test.jpg','wb') as file: 
    file.write(r.content)

'%E8%81%96%E9%97%98%E5%A3%AB%E6%98%9F%E7%9F%A2'

In [115]:
#get_img(ROOT_URL  + "/image/10679272/20/1","./data/image_10679272_20_1")

In [74]:
ITEM_URL = "https://www.1999.co.jp/10679272"
ITEM_IMG_URL ="https://www.1999.co.jp/image/10679272/20/1"
img_soup = get_soup(ITEM_IMG_URL, SLEEP_SEC)

In [98]:
#img_url = img_soup.find("div", class_="TagCenter").find("img")["src"]
img_url = img_soup.find("table", id="imgTarget").find("img")["src"]
img_url
get_img(ROOT_URL + img_url, "data/" + img_url.replace("/","_"))


In [160]:
for div in item_soup.find("div", id="masterBody_pnlItemExp").findAll("div"):
    if  "●全高：" in div.text:
        hight = div.text.split("●全高：")[1].split("\n")[0]
        print(hight)

約160mm
約160mm


In [152]:
 s = item_soup.find("div", id="masterBody_pnlItemExp")